In [ ]:
from glob import glob
import pandas as pd
DATA_DIR = "/.cache/steel_dataset/wanjuan/part-003756-a894b46e.jsonl"
from tqdm import tqdm
import json
import re
import random

In [ ]:
def format_example(item, cot=False):
    example = item["q_main"] + "\n"
    choice_map = {"A":"option_a", "B":"option_b", "C":"option_c", "D":"option_d"} 
    for choice in ["A","B","C","D"]:
        if item[choice_map[choice]] != "":
            example += f'{choice}. {item[choice_map[choice]]}\n'
    # 非cot
    if not cot:
        answer = f'{item["std_ans"]}。{item["answer_detail"]}'
    # cot
    else:
        answer = f'{item["answer_detail"]}'
        pattern = r"故选.*"
        answer = re.sub(pattern, '', answer, flags=re.DOTALL)
        answer = answer + f'答案为{item["std_ans"]}'
    return example, answer

all_data = []
no_explain_counter = 0
cot = True
with open(DATA_DIR, 'r') as file:
    for idx, line in enumerate(file):
        # 解码JSON对象
        json_obj = json.loads(line)
        if json_obj["std_ans"]=="" or len(json_obj["answer_detail"])<10:
            # 200w数据这块continue
            choice, answer = format_example(json_obj, False)
            instruct = f"以下是一道单选题:\n{choice}请给出答案。\n"
            a_data = {
                    "instruction": instruct,
                    "input": "",
                    "output": answer
                }
            all_data.append(a_data)
        else:
            # 处理json_obj
            choice, answer = format_example(json_obj, cot)
            if not cot:
                instruct = f"以下是一道单选题:\n{choice}请给出答案。\n"
            # cot
            else:
                instruct = f"以下是一道{json_obj['q_type']}:\n{choice}请先给出解释再给出答案。\n"
            a_data = {
                    "instruction": instruct,
                    "input": "",
                    "output": answer
                }
            # print(a_data)
            all_data.append(a_data)

In [ ]:
print(len(all_data))

In [ ]:
print("filter before:", len(all_data))
filted_all_data = [x for x in all_data if "略" not in x["output"]]
print("filter after:", len(all_data))

In [ ]:
import random
few_data = random.sample(filted_all_data, 200000)
print(len(few_data))
print(few_data[100])

In [ ]:
import json
import os
file_path = '/.cache/sft_data/llamafactory_input/wanjuan_exam_399w.jsonl'

directory = os.path.dirname(file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"目录{directory} 不存在，已创建")
else:
    print(f"目录{directory} 已存在")

with open(file_path, 'w', encoding="utf-8") as f:
    for item in filted_all_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')